In [1]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv("train_ready_for_model.csv")

In [4]:
test = pd.read_csv("test_ready_for_model.csv")

In [5]:
train.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20,target_value,ID
0,4.175829,-0.913514,-0.774966,-3.194906,-0.187935,0.885000,-4.232843,-1.348705,3.309659,3.307302,...,-1.474777,1.286419,1.660316,-0.573375,-1.804728,0.297913,0.222736,1.147765,1,142882
1,-1.691200,-0.387239,-0.076539,-0.611680,0.473701,0.107667,-0.721878,-0.082401,0.061310,-1.103078,...,0.048245,0.115520,0.095146,-0.062336,-0.361427,-0.085127,0.036760,0.228798,0,116073
2,8.961550,-0.924506,-0.542125,-4.756978,1.456443,1.538814,-1.287939,-1.362948,3.428058,5.032791,...,-0.423228,0.347153,0.201813,0.342466,-0.163954,-0.225152,0.254356,-0.275367,0,40727
3,2.167607,-3.490266,0.663704,3.511780,2.447511,-0.229028,2.988597,0.167085,-0.664976,0.299895,...,0.230584,0.018007,-0.592949,-1.697078,1.086859,-1.985715,-0.208183,-0.835496,0,96753
4,-1.668510,-0.340204,-0.095412,-0.772138,0.570506,0.146925,-0.862205,-0.099139,0.065471,-1.259335,...,0.076081,0.109953,0.091492,-0.039760,-0.380755,-0.087354,0.044924,0.233986,0,105319


In [6]:
test.head()

,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,...,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20,target_value,ID
0,4.348331,-1.038153,-0.818104,-3.434128,-0.259648,0.995374,-4.221248,-1.643019,4.180365,4.322214,...,-1.535099,1.436590,1.842944,-0.473695,-2.054858,0.469429,0.206340,1.282464,0,64252
1,1.382517,-0.258735,0.135192,-2.012257,1.879665,0.395088,1.123130,0.076308,-0.696915,-1.781094,...,-0.056691,0.207165,0.095370,-0.516819,-0.390286,-0.008873,-0.219549,0.239203,0,31278
2,-1.673492,-0.334366,-0.082494,-0.750899,0.570795,0.142003,-0.826247,-0.076613,0.061630,-1.246865,...,0.058923,0.113530,0.097256,-0.080508,-0.374379,-0.099072,0.003729,0.244663,0,148245
3,-1.067577,0.100077,-0.142491,-0.683594,0.672578,0.132939,-0.955903,-0.010377,-0.005110,-1.110933,...,0.292594,-0.246974,0.040136,0.018848,0.800290,0.663549,0.107858,-0.137776,0,133657
4,-1.423364,-0.070934,-0.148320,-1.059737,0.706684,0.209845,-1.146108,-0.072422,-0.054328,-1.589526,...,0.051317,0.151240,0.109996,-0.083896,-0.416487,-0.112402,0.072274,0.244294,0,68463


In [16]:
from sklearn import metrics,model_selection
 #from sklearn import cross_validation,
from sklearn.metrics import accuracy_score,f1_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [19]:
test_act=pd.read_csv("Result Evaluation.csv")
test_act.head()
test_act['Target_Class']=np.where(test_act['TARGET']>0,1,0)
test_act.drop(columns='TARGET',inplace=True)
test_act.head()

,ID,Target_Class
0,64252,0
1,31278,0
2,148245,0
3,133657,0
4,68463,0


In [22]:
#Define target and ID columns:
target = 'target_value'
IDcol = ['ID']
from sklearn import model_selection, metrics
from sklearn.metrics import classification_report

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    kfold = model_selection.KFold(n_splits=10, random_state=7)

    #Perform cross-validation:
    cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='f1')
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    print(classification_report(dtrain[target].values, dtrain_predictions))
    #print("Train CV score : ", cv_train_score)
    #print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])   
    print(classification_report(test_act['Target_Class'].values, dtest[target]))
    print("F1 Score:",metrics.f1_score(test_act['Target_Class'].values, dtest[target]))
    #cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=kfold, scoring='accuracy')
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)



In [23]:
from sklearn.linear_model import LogisticRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = LogisticRegression(verbose=1)
modelfit(alg1, train, test, predictors, target, IDcol, 'Logreg.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#print(coef1)
#coef1.plot(kind='bar', title='Model Coefficients')

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]
Model Report
              precision    recall  f1-score   support

           0       0.91      1.00      0.95     12715
           1       0.35      0.01      0.01      1284

   micro avg       0.91      0.91      0.91     13999
   macro avg       0.63      0.50      0.48     13999
weighted avg       0.86      0.91      0.87     13999

              precision    recall  f1-score   support

           0       0.91      1.00      0.95      5282
           1       0.30      0.01      0.01       524

   micro avg       0.91      0.91      0.91      5806
   macro avg       0.61      0.50      0.48      5806
weighted avg       0.86      0.91      0.87      5806

F1 Score: 0.011235955056179777


In [24]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=750,max_depth=30,verbose=1,min_samples_leaf=1,min_samples_split=5, random_state=1,n_jobs=-1)
modelfit(rf, train, test, predictors, target, IDcol, 'Rf.csv')

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:   14.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 750 out of 750 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 750 out of 750 | elapsed:   28.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 750 out of 750 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Us


Model Report
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     12715
           1       1.00      0.93      0.97      1284

   micro avg       0.99      0.99      0.99     13999
   macro avg       1.00      0.97      0.98     13999
weighted avg       0.99      0.99      0.99     13999



[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 750 out of 750 | elapsed:    0.3s finished


              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5282
           1       0.36      0.13      0.20       524

   micro avg       0.90      0.90      0.90      5806
   macro avg       0.64      0.56      0.57      5806
weighted avg       0.87      0.90      0.88      5806

F1 Score: 0.19553072625698323


In [25]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
dtree = DecisionTreeClassifier(criterion='gini',max_depth=30)
adb = AdaBoostClassifier(base_estimator= dtree,
        n_estimators=50,learning_rate=0.05,random_state=42)
modelfit(adb, train, test, predictors, target, IDcol, 'ADB.csv')


Model Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     12715
           1       1.00      1.00      1.00      1284

   micro avg       1.00      1.00      1.00     13999
   macro avg       1.00      1.00      1.00     13999
weighted avg       1.00      1.00      1.00     13999

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      5282
           1       0.23      0.27      0.25       524

   micro avg       0.85      0.85      0.85      5806
   macro avg       0.58      0.59      0.58      5806
weighted avg       0.86      0.85      0.86      5806

F1 Score: 0.24536628420123568
